# Preprocessing, Modeling

In [ ]:
# Importing our libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import geodesic
import datetime as dt

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import (roc_auc_score, confusion_matrix, recall_score, precision_score, 
                             roc_curve, f1_score, plot_confusion_matrix)

from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier, plot_importance
import shap

In [ ]:
# Ensuring notebook remains tidy
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting our seaborn style and palette
sns.set_style('darkgrid')
sns.set_palette('icefire')

In [ ]:
# Creating a custom function to kick-start the EDA process
def eda_clean(df):
    print('Dataset Statistics:')
    print(f'Shape of dataframe: {df.shape}')
    print('--------------------------------------')
    print(f'Null values in dataframe: {df.isna().sum().sum()}')
    print('--------------------------------------')
    print(f'% of Null values in dataframe: {round(((df.isna().sum().sum())/(df.shape[0])) * 100, 2)}%')
    print('--------------------------------------')
    print(f"Total duplicate rows: {df[df.duplicated()].shape[0]}")
    print('--------------------------------------')
    print(f"% duplicate rows: {round(df[df.duplicated()].shape[0] / df.shape[0] * 100, 2)}%")
    print(f'\nColumn names: {df.columns}')
    print('\nVariable Types')
    print(f"Columns Count: \n{df.dtypes.value_counts()}")

In [ ]:
# Import our tidied train dataset
train_transformed = pd.read_csv('../data/cleaned/train_tidied.csv')

In [ ]:
# Dropping redundant columns
train_transformed.drop(columns=['date', 'addressnumberandstreet', 'year_month', 'station', 'station_ref', 'year'], inplace=True)

In [ ]:
# Dropping null and nan columns
train_transformed.dropna(inplace=True)

In [ ]:
# A quick snapshot of our final dataframe
train_transformed.head()

In [ ]:
# Defining X and y variables for our train split
X = train_transformed.drop(columns='wnvpresent')
y = train_transformed['wnvpresent']

In [ ]:
# Do a train-test split of 70-30 for train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    stratify=y, # Because this is an unbalanced dataset
                                                    random_state=42)

**Polynomial Features**

We have accounted for the interactions between different features. Model results were not very different.

In [ ]:
# # generates the full polynomial feature table
# pf = PolynomialFeatures(include_bias=False, degree=2)
# X_train_pf = pf.fit_transform(X_train)

# # Adds appropriate feature names to all polynomial features
# X_train_pf = pd.DataFrame(X_train_pf, columns=pf.get_feature_names(X_train.columns))

**Solving the problem with an unbalanced dataset** 

In classification problems, balancing your data is absolutely crucial. Data is said to be imbalanced when instances of one class outnumber the other(s) by a large proportion. One of the sampling techniques that we have decided to employ is **SMOTE** (Synthetic Minority Over-Sampling Technique).

Just like the name suggests, the technique generates synthetic data for the minority class. SMOTE proceeds by joining the points of the minority class with line segments and then places artificial points on these lines. Here's how it works:

* Choose a minority class input vector
* Find its k nearest neighbors (k_neighbors is specified as an argument in the SMOTE() function)
* Choose one of these neighbors and place a synthetic point anywhere on the line joining the point under consideration and its chosen neighbor
* Repeat the steps until data is balanced

In [ ]:
# create function to run model with standard scaling, SMOTE sampling, 
# with an option to run grid search, model and print results

def run_model(mod, mod_params={}, grid_search=False):
    
    # Initial dictionary to hold model results
    results = {}
    
    pipe = Pipeline([
            ('ss', StandardScaler()),
            (mod, models[mod])
            ])
    
    if grid_search:
        # Instantiate list to store gridsearch results
        gs = GridSearchCV(pipe, param_grid=mod_params, cv=5, verbose=1, scoring='roc_auc', n_jobs=-1)
        gs.fit(X_train, y_train)
        pipe = gs
    else:
        pipe.fit(X_train, y_train)
        

    # Retrieve metrics
    predictions = pipe.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(X_test)[:,1]
    y_train_pred_prob = pipe.predict_proba(X_train)[:,1]
    auc_scores = cross_val_score(pipe, X_train, y_train,  scoring='roc_auc', cv = 5)

    results['model'] = mod
    
    results['train_auc_cv'] = auc_scores.mean()
    
    results['f1'] = f1_score(y_test, predictions)
    results['recall'] = recall_score(y_test, predictions)        # % OF ACTUAL positives that are CORRECTLY predicted
    results['precision'] = precision_score(y_test, predictions)  # % OF positives that are CORRECTLY predicted

    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    results['auc_diff'] = results['train_auc'] - results['test_auc']

    if grid_search:
        gs_list.append(results)
        print('### BEST PARAMS ###')
        display(pipe.best_params_)
        
    else:
        init_list.append(results)

    return pipe

In [ ]:
%%time

X_train_copy = X_train.astype(float).copy()
y_train_copy = y_train.astype(float).copy()

# create loop to run SMOTE sampling and compare the modelling outcomes with and without it
for k in ['No', 'SMOTE']:
    print('\nMethod Used: {}'.format(k + ' sampling'), "-" * 100)
        
    print('\nClass Balance BEFORE')
    display(y_train.value_counts(normalize=True))
    print('Number of rows: {}'.format(y_train.shape[0]))
  
    # instiantiate the models
    methods = {'SMOTE': SMOTE(random_state=42)}
    
    if k == 'SMOTE':
        mthd = methods[k]
        X_train, y_train = mthd.fit_resample(X_train, y_train)

    print('\nClass Balance AFTER')
    display(y_train.value_counts(normalize=True))
    print('Number of rows: {}'.format(y_train.shape[0]),'\n')
    
    
    # Instiantiate models
    models = {'lr': LogisticRegression(max_iter=5_000, random_state=42, solver='saga'),
#               'rf': RandomForestClassifier(random_state=42),
#               'gb': GradientBoostingClassifier(random_state=42),
#               'dt': DecisionTreeClassifier(random_state=42),
#               'et': ExtraTreesClassifier(random_state=42),
#               'ada': AdaBoostClassifier(random_state=42),
#               'svc': SVC(random_state=42, probability=True),
              'xgb': XGBClassifier(random_state=42, 
                              objective='binary:logistic', 
                              verbosity=1, n_jobs=-1)
            }

    # Instantiate lists to store results
    init_list = []
    gs_list = []

    for m in models:
        run_model(m)
    result_df = pd.DataFrame(init_list).sort_values(by=["f1"], ascending=False).reset_index(drop=True)
    display(result_df)

    X_train = X_train_copy
    y_train = y_train_copy

## Selection of our final model (XGBoost)

Due to the unbalanced nature of this dataset, the main metric on our radar is the ROC AUC. **XGBoost** fares scores pretty well on the validation set and requires less computational input compared to **ADABoost**.

We will create a pipeline and tune some hyperparameters below.

In [ ]:
# Creating a pipeline model for StandardScaler, SMOTE and XGBoost
xgb_pipe = Pipeline([
        ('ss', StandardScaler()),
        ('xgb', XGBClassifier(random_state=42, 
                              objective= 'binary:logistic', 
                              verbosity=1, n_jobs=-1))
    ])
# Parameters
xgb_params = {  'xgb__learning_rate': [0.1],
                'xgb__max_depth': [40],
                'xgb__min_child_weight': [25],
                'xgb__gamma': [0.85],
                'xgb__subsample': [1],
                'xgb__scale_pos_weight': [63],
                'xgb__n_estimators': [49]}

In [ ]:
%%time

xgb_best = run_model('xgb', xgb_params, grid_search=True)

In [ ]:
# create table to show the predictors with the highest importance (i.e. strongest predictors of the presence of WNV)
feature_names = X.columns
coefficients = pd.DataFrame(np.squeeze(xgb_best.best_estimator_.named_steps["xgb"].feature_importances_),
                            columns=['Importances']) 
features = pd.DataFrame(np.squeeze(feature_names), columns=['Features']) 
df = pd.concat([features, coefficients], axis='columns').sort_values('Importances', ascending=False)
df

In [ ]:
y_pred_prob = xgb_best.best_estimator_.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Plot ROC curve
plt.figure(figsize=(16, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve, ROC AUC Score: {}'.format(roc_auc_score(y_test, y_pred_prob).round(3)))
plt.show()

## Tidying ```test``` data

We will need to import our test data and carry out the pre-requisite pre-processing steps. We have created custom functions for this bit of the codebook.
* Initial cleaning (Dropping address, block, street, addressaccuracy columns)
* Date feature engineering (Extracting year, month, week, day variables)
* Dummify our ```cluster``` column
* One-hot encoding mosquito species
* Determining if a coordinate is closer to Station 1 or Station 2
* Final cleaning (Dropping the remaining unnecessary columns)
* Generating predictions from our best model

After applying each function, we will map it to a new variable to ensure the process flows smoothly.

In [ ]:
# Read train and test datasets
test = pd.read_csv('../data/cleaned/test_tidied.csv', parse_dates=['date'])

# Converting columns to lowercase for standardisation
test.columns = test.columns.str.lower()

In [ ]:
# Previewing our test dataset
eda_clean(test)

In [ ]:
def initial_clean(df):
    df.drop(columns=['address', 'block', 'street', 'addressaccuracy'], inplace=True)
    return df

In [ ]:
test_iter1 = initial_clean(test)

In [ ]:
def date_feature_engineering(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['week'] = df['date'].dt.isocalendar().week
    df['day'] = df['date'].dt.day
    df['year_month'] = df['date'].dt.strftime("%Y-%m")

    return df

In [ ]:
test_iter2 = date_feature_engineering(test_iter1)

In [ ]:
test_iter3 = pd.get_dummies(test_iter2, columns=['cluster'])

In [ ]:
# We can exclude Unsepcified species from this selection
species_of_interest = ['RESTUANS', 'PIPIENS', 
                       'SALINARIUS', 'TERRITANS', 'TARSALIS', 'ERRATICUS'
                      ]

# create loop to create dummified variables to indicate presence of Pipiens & Restuans species
def dummify_species(df):
    for i in species_of_interest:
        df[i] = df['species'].apply(lambda x: 1 if i in x else 0)
    
    return df

In [ ]:
test_iter4 = dummify_species(test_iter3)

In [ ]:
def determine_station(lat, long):
    station1 = (41.995, -87.933) #Chicago O'Hare Tntl Airpot -> lat: 41.995 long: -87.933
    station2 = (41.786, -87.752) #Chicago Midway Tntl Airpot -> lat: 41.786 long: -87.752
    coordinates = (lat, long)
    
    return 1 if geodesic(coordinates, station1) < geodesic(coordinates, station2) else 2

In [ ]:
%%time

# Apply function to map our function
test_iter4['station_ref'] = test_iter4.apply(lambda i: determine_station(i['latitude'], i['longitude']), axis=1)

In [ ]:
# Import tidied weather dataset
weather_tidied = pd.read_csv('../data/cleaned/weather_tidied.csv', parse_dates=['date'])

In [ ]:
test_iter5 = test_iter4.merge(weather_tidied, how='left', left_on=['station_ref', 'date'], right_on=['station', 'date'])
test_iter5.drop(columns=["station"], inplace=True)

In [ ]:
def final_clean(df):
    df.drop(columns=['species', 'trap', 'date', 'addressnumberandstreet', 
                     'latitude', 'longitude', 
                     'day', 'coord',
                     'year_month', 'station_ref', 'year'], inplace=True)
    
    return df

In [ ]:
test_iter6 = final_clean(test_iter5)

In [ ]:
def predict_best(df, model_best):
    df_id = df['id']

    df_base = df.drop(columns=['id'])
    
    # generate predictions
    y_pred = model_best.best_estimator_.predict(df_base)  

    # load prediction into dataframe
    df_pred = pd.DataFrame(y_pred, columns=['wnvpresent']) 

    # Merge id with predictions into one dataframe
    df_final = pd.concat([df_id, df_pred], axis='columns')
    return df_final

def predict_proba_best(df, model_best):
    df_coord = test[['latitude', 'longitude']]
    
    df_base = df.drop(columns=['id'])
    
    # generate predictions
    y_pred = model_best.best_estimator_.predict_proba(df_base)  

    # load prediction into dataframe
    df_pred = pd.DataFrame(np.squeeze(y_pred)) 

    # Merge id with predictions into one dataframe
    df_final = pd.concat([df_coord, df_pred], axis='columns')
    return df_final

In [ ]:
# generate predictions
pred = predict_best(test_iter6, xgb_best)

In [ ]:
pred.rename(columns={'id': 'Id', 'wnvpresent':'WnvPresent'}, inplace=True)

In [ ]:
pred = pred.set_index('Id')

In [ ]:
pred.to_csv('../kaggle/submission.csv')

**Kaggle Submission Score**

**Thoughts**: We believe further improvements can be made to the model (if given enough time). Here are some of our considerations:
* There could be a bias for certain traps (Given that a particular trap may always capture more mosquitos, etc)
* Possibility of a daily multiplier given an array of outbreaks; Outbreaks will possibly influence the days before and after it (They tend to cluster in the dimension of time too)

![Our Kaggle submission](../images/kaggle_submission.png "Our Kaggle submission")